In [4]:
import string;
from gensim.models import doc2vec
import numpy as np
import nltk

# We import sklearn.
import sklearn
from sklearn.manifold import TSNE

# Random state.
RS = 30145621

# We'll use matplotlib for graphics.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib
%matplotlib inline
import csv
import random

import warnings
warnings.filterwarnings('ignore')

import sys
import gc
import pandas as pd
import pickle

csv.field_size_limit(sys.maxsize)

131072

In [5]:
from datetime import timedelta, date

years = [7,10,11,12,13,14,15,16]


def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)


dates = {}        
for year in years:
    days = []
    year = year + 2000
    start = date(year, 1, 1)
    end = date(year, 12, 31)
    for dt in daterange(start, end):
        days.append(dt.strftime("%m-%d-%Y"))
    dates[year] = days

In [3]:
import pickle

docfile = open('documents.pkl', 'rb')
tokens = pickle.load(docfile)

idfile = open('bgt_500.pkl', 'rb')
job_IDs = pickle.load(idfile)

In [4]:
def create_tagged_objects(tokens):
    """Convers tokens to genism tagged documents"""
    tagged_docs = []
    for i in range(len(tokens) - 1):
        tagged_doc = doc2vec.TaggedDocument(words=tokens[i], tags=[i])
        tagged_docs.append(tagged_doc)
    return tagged_docs


tagged_docs = create_tagged_objects(tokens)    

In [5]:
model = doc2vec.Doc2Vec(alpha=0.05, min_alpha=0.05, vector_size=100, window=8, min_count=5, workers=4)


print('Building vocab')
model.build_vocab(tagged_docs)
print('Vocab length: ' + str(len(model.wv.vocab)))

len(model.docvecs)
epochs = 20
factor = 0.9
for epoch in range(epochs):
    # train epoch by epoch
    print('Training epoch %d' % epoch)
    random.shuffle(tagged_docs)
    model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)
    model.alpha *= factor  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

Building vocab
Vocab length: 168764
Training epoch 0
Training epoch 1
Training epoch 2
Training epoch 3
Training epoch 4
Training epoch 5
Training epoch 6
Training epoch 7
Training epoch 8
Training epoch 9
Training epoch 10
Training epoch 11
Training epoch 12
Training epoch 13
Training epoch 14
Training epoch 15
Training epoch 16
Training epoch 17
Training epoch 18
Training epoch 19


In [7]:
model.save('model_500')

In [6]:
model = doc2vec.Doc2Vec.load('model_500')

In [8]:
model.wv.most_similar('python')

[('perl', 0.8188003897666931),
 ('scala', 0.7101612687110901),
 ('mysql', 0.6844235062599182),
 ('java', 0.6597845554351807),
 ('matlab', 0.6268130540847778),
 ('fortran', 0.6162153482437134),
 ('vbscript', 0.6119515895843506),
 ('rubi', 0.5884727835655212),
 ('angularj', 0.5854419469833374),
 ('javascript', 0.5740393400192261)]

In [9]:
model.wv.most_similar('softwar')

[('15001daq', 0.5243683457374573),
 ('firmwar', 0.5080764293670654),
 ('winfit', 0.5080638527870178),
 ('assetwis', 0.4921538233757019),
 ('cablescan', 0.47701284289360046),
 ('sofwar', 0.47694694995880127),
 ('smftware', 0.46843552589416504),
 ('java', 0.466549277305603),
 ('terqitori', 0.46324294805526733),
 ('iseri', 0.45917296409606934)]

In [18]:
model.wv.most_similar('communication')

[('readback', 0.5105516314506531),
 ('ability_', 0.4897938370704651),
 ('partssafeti', 0.4727225601673126),
 ('systemfriendli', 0.47023242712020874),
 ('varazdin', 0.46668708324432373),
 ('friendly_', 0.4444960951805115),
 ('interiorhr', 0.4225849211215973),
 ('chaoyang', 0.421771764755249),
 ('taichung', 0.42097795009613037),
 ('timisoara', 0.4202026426792145)]

In [13]:
model.wv.most_similar('medical')

[('planningcoordin', 0.42966702580451965),
 ('yergeau', 0.40698906779289246),
 ('send', 0.4037514925003052),
 ('calul', 0.40098243951797485),
 ('huber', 0.39607077836990356),
 ('stenographi', 0.3959091305732727),
 ('emial', 0.39531877636909485),
 ('evin', 0.3887404799461365),
 ('alliancestaff', 0.38652896881103516),
 ('protim', 0.38541489839553833)]

In [19]:
vectors = []
for i in range(len(model.docvecs)-1):
    vectors.append(model.docvecs[i])

In [20]:
def scatter(x):
    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40)
    ax.axis('on')
    ax.axis('tight')
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    return f, ax, sc

In [ ]:
# We create a scatter plot with perplexity = 50
docs_proj_50 = TSNE(random_state=RS, perplexity=50).fit_transform(vectors)
scatter(docs_proj_50)

In [ ]:
# We create a scatter plot with perplexity = 30
docs_proj_30 = TSNE(random_state=RS, perplexity=30).fit_transform(vectors)
scatter(docs_proj_30)

In [ ]:
# We create a scatter plot with perplexity = 5
docs_proj_5 = TSNE(random_state=RS, perplexity=5).fit_transform(vectors)
scatter(docs_proj_5)